# Cross-Fitting and $p$-Value Combination

In this tutorial, we will demonstrate how to implement cross-fitting and $p$-value combination by using `Crosser`. The `Crosser` object also provides a user-friendly interface to conduct nested cross-validation (CV) that results in a more stable result under small sample sizes.

We will use the `diabetes` data set again for demonstration. The following code prepares the data for the tutorial.

In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
X, y = datasets.load_diabetes(
    return_X_y = True,
    as_frame = True,
    scaled = False)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size = .5)

## `Crosser` Overview

Briefly speaking, `Crosser` is an object that can conduct nested cross-validation (CV) to improve the stability of learning result. Suppose we have split the a data set $Z$ into 5 folds, say $Z^{(1)}, Z^{(2)}, Z^{(3)}, Z^{(4)}, Z^{(5)}$. For each $k \in \{1,2,3,4,5\}$, we set $Z_\text{train}^{(k)} = \{Z_l\}_{l \neq k}$ and $Z_\text{test}^{(k)} = \{Z_k\}$. Then $Z_\text{train}^{(k)}$ is used to train $\widehat{f}^{(k)}$ and $Z_\text{test}^{(k)}$ is used to evaluate the performance of $\widehat{f}^{(k)}$. Note that when training $\widehat{f}^{(k)}$ an inner CV loop may be used.

To understand `Crosser` more concretely, let use create a cross-validator via `KFold`. The cv can help us to create the training and test indices for the 5-fold CV. An important thing here is that a specific `random_state` must be given. Otherwise, `sial` may yield a wrong result when making statistical inferences.

In [2]:
from sklearn.model_selection import KFold
cv = KFold(
    n_splits = 5,
    shuffle = True,
    random_state = 1)

Now we initialize a `Crosser` object. When initializing, an estimator and a cross-validator must be given. The estimator can be a usual `scikit-learn` estimator or a `GridSearchCV` object as in this example. The `GridSearchCV` object runs an inner CV loop to find an optimal tuning parameter value for each $\widehat{f}^{(k)}$. The `fit` method trains $\widehat{f}^{(1)}$, $\widehat{f}^{(2)}$, ..,$\widehat{f}^{(5)}$ on their corresponding training sets.

In [3]:
from sial import Crosser
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
learner = Crosser(
    GridSearchCV(
        estimator = RandomForestRegressor(), 
        param_grid = {
            "max_features": [3, 6, 9]}),
    cv = cv)
_ = learner.fit(X, y)

To extract the trained estimators, we can use the `estimator_` attribute:

In [4]:
len(learner.estimators_)

5

The overall performances of these trained estimators can be shown by the `summarize` method. Three types of error scores are reported. The `val_score` is the validation error extracted from the `GridSearchCV`. The `train_score` and `test_score` are calculated by `Crosser`. By default, regression tasks use `r2` scorer and classification tasks uses `accuracy`. The scoring method can be explicitly specified via the `scoring` argument when initializing a `Crosser`. Availabe scoring methods can be found [here](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values). 

In [5]:
learner.summarize()

Crosser Summary (cross_fit=True, combine=False)
 + Estimator: GridSearchCV
 + Cross-Validator: KFold (n_folds=5, n_repeats=1)
 + Train/Test Scorer: R2 (reverse=False)


,val_score,train_score,test_score
repeat,,,
0,0.438521,0.923787,0.425175


By default, the `summarize` method averages the error scores. It is also possible to show the error scores for each fold by setting `cross_fit = False`. We can see that the test errors are quite different across folds.

In [6]:
learner.summarize(
    cross_fit = False)

Crosser Summary (cross_fit=False, combine=False)
 + Estimator: GridSearchCV
 + Cross-Validator: KFold (n_folds=5, n_repeats=1)
 + Train/Test Scorer: R2 (reverse=False)


,,,val_score,train_score,test_score
split,repeat,fold,,,
0,0,0,0.480746,0.928988,0.392548
1,0,1,0.467891,0.925515,0.300636
2,0,2,0.418019,0.922168,0.445106
3,0,3,0.421659,0.921233,0.452291
4,0,4,0.404291,0.921029,0.535294


The `Crosser` also has a `predict` method for prediction. For a regressor, by default, it makes predictions by averaging the predictions made by all the trained estimators:

In [7]:
learner.predict(
    X.iloc[:5,:])

array([179.688,  80.214, 158.082, 192.374, 119.66 ])

When the estimator is a classifier, the prediction is based on majority voting. If we hope to make predictions based on a model trained on specific split, the argument `split` can be used:

In [8]:
learner.predict(
    X.iloc[:5,:],
    split = 0)

array([179.86,  78.37, 156.75, 201.22, 105.67])

## Inference with Cross-Fitting via `Crosser`

Now we demonstrate how to implement cross-fitting strategy to improve both statistical power and stability of a test with the help of `Crosser`. In breif, the cross-fitting atrategy try to integrate the test result on each fold: 

+ When the null distribution is constructed by `resampling` method, the integration is based on Algorithm 4 of Tansey et al (2022);
+ When the null distribution is constructed by `normality` and `permutation` method, the integration is based on Algorithm 3 of Williamson et al. (2023).

In this example, the conditional predictive impact (CPI) will be used to test the significance of `sex`. Hence, a sampler for `sex` is necessary. Note that the sampler must be trained with the same cross-validator as we specified for training the learner. Otherwise, `sial` cannot perform cross-fitting correctly.

In [9]:
from sklearn.ensemble import RandomForestClassifier
removal = "sex"
sampler = Crosser(
    GridSearchCV(
        estimator = RandomForestClassifier(), 
        param_grid = {
            "max_features": [3, 6, 9]}),
    cv = cv)
_ = sampler.fit(
    X.drop(removal, axis = 1), 
    X[removal])

To implement cross-fitting, we replace usual estimators by their corresponding `Crosser` objects when initializing the `Inferer`. The `summarize` method will integrate the results via cross-fitting by default:

In [10]:
from sial.inferer import CIT
cpi = CIT(
    learner, 
    sampler,
    removal,
    "CPI",
    n_copies = 100)
_ = cpi.infer(X, y)
cpi.summarize()

Inferer Summary (cross_fit=True, combine=False)
 + Method: CPI (double_split=None, perturb_size=None)
 + Null Distribution: Normality (n_copies=100, n_permutations=None)
 + Loss Function: Mean Squared Error (reverse=False)


,size,estimate,std_error,p_value
removal,,,,
sex,442,54.395756,17.728132,0.001076


To inspect the result on each fold, we can set `cross_fit = False`. We can see that these results are unstable bacause of the small sample sizes of the `diabetes` data.

In [11]:
cpi.summarize(
    cross_fit = False)

Inferer Summary (cross_fit=False, combine=False)
 + Method: CPI (double_split=None, perturb_size=None)
 + Null Distribution: Normality (n_copies=100, n_permutations=None)
 + Loss Function: Mean Squared Error (reverse=False)


size    estimate  std_error   p_value
removal split repeat fold                                       
sex     0     0      0       89   21.346994  31.760977  0.250756
        1     0      1       89  116.543331  36.613575  0.000729
        2     0      2       88   -1.672984  35.795756  0.518639
        3     0      3       88   43.603011  44.527297  0.163730
        4     0      4       88   92.158430  49.508939  0.031340

## Inference with Both Cross-Fitting and $p$-Value Combination

Another way to improve the stability of testing is using $p$-value combination. In this example, we use both cross-fitting and $p$-value combination. We plan to conduct 5-fold cross-validation four times. Hence, `RepeatedKFold` is used as a cross-validator:

In [12]:
from sklearn.model_selection import RepeatedKFold
cv = RepeatedKFold(
    n_splits = 5,
    n_repeats = 4,
    random_state = 1)

In `sial`, `n_splits` is used to denote the number of all splits and the number of folds will be represented by another variable called `n_folds`. Hence, the specified `n_splits = 5` and `n_repeats = 4` in this example results in `n_splits = 20`, `n_folds = 5`, and `n_repeats = 4` stored in both `Crosser` and `Inferer`.

Then we train a learner and a sampler with the `RepeatedKFold` cross-validator and conduct CPI for `sex`. By default, the `summarize` method implement several p-value combination methods that aggregate the $p$-values obtained in the four replications.

In [ ]:
learner = Crosser(
    GridSearchCV(
        estimator = RandomForestRegressor(), 
        param_grid = {
            "max_features": [3, 6, 9]}),
    cv = cv)
sampler = Crosser(
    GridSearchCV(
        estimator = RandomForestClassifier(), 
        param_grid = {
            "max_features": [3, 6, 9]}),
    cv = cv)
_ = learner.fit(X, y)
_ = sampler.fit(
    X.drop(removal, axis = 1), 
    X[removal])

In [ ]:
cpi = CIT(
    learner, 
    sampler,
    removal,
    "CPI",
    n_copies = 100)
_ = cpi.infer(X, y)
cpi.summarize()

To see the $p$-value in each replication, we can set both `combine = False` in the `summarize` method. Note that these $p$-values are calculated based on cross-fitting. Even after cross-fitting, the $p$-values still differ slightly across replications. That is why $p$-value combination could be helpful for enhencing reproducibility.

In [ ]:
cpi.summarize(
    combine = False)

If users are interested in the $p$-value on each split, we can set `combine = False` and `cross_fit = False`. We can see that the split-level results are indeed unstable.

In [ ]:
cpi.summarize(
    cross_fit = False,
    combine = False)

Breifly speaking, `cross_fit` controls aggregating results over `n_folds` given a repeat and `combine` controls combining results over `n_splits`. If both `cross_fit` and `combine` are `True`, the `summarize` method first aggregates the results of folds for each replication and then do $p$-value combination based on the results made by first step.

Finally, it is possible to combine all $p$-values without doing cross-fitting first by setting `combine = True` and `cross_fit = False`. This approach might result in conservative results for some p-value combination methods. However, so far it is hard to say which method is more reliable. The relative performances of these combination methods require further empirical evaluations.

In [ ]:
cpi.summarize(
    cross_fit = False,
    combine = True)

## References

Tansey, W., Veitch, V., Zhang, H., Rabadan, R., & Blei, D. M. (2022). The holdout randomization test for feature selection in black box models. Journal of Computational and Graphical Statistics, 31(1), 151–162. doi: 10.1080/10618600.2021.1923520

Williamson, B. D., Gilbert, P. B., Simon, N. R., & Carone, M. (2023). A general framework for inference on algorithm-agnostic variable importance. Journal of the American Statistical Association, 118(543), 1645–1658. doi: 10.1080/01621459.2021.2003200